<a href="https://colab.research.google.com/github/gauravbyte/DL_Assignment_3/blob/master/DL_ASSIGNMENT_3_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 20.5 MB/s eta 0:00:00


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np

# Instantiates the device to be used as GPU/CPU based on availability
device_gpu = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
df_train = pd.read_csv("/content/drive/MyDrive/DL_ASSIGNMENT_3/aksharantar_sampled/hin/hin_train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/DL_ASSIGNMENT_3/aksharantar_sampled/hin/hin_test.csv")
df_valid = pd.read_csv("/content/drive/MyDrive/DL_ASSIGNMENT_3/aksharantar_sampled/hin/hin_valid.csv")

In [9]:
print(df_train)
print(df_test)
print(df_valid)

       shastragaar शस्त्रागार
0          bindhya   बिन्द्या
1        kirankant   किरणकांत
2      yagyopaveet  यज्ञोपवीत
3          ratania    रटानिया
4       vaganyache  वागण्याचे
...            ...        ...
51194        toned       टोंड
51195   mutanaazaa   मुतनाज़ा
51196    asahmaton    असहमतों
51197    sulgaayin   सुलगायीं
51198  anchuthengu  अंचुतेंगु

[51199 rows x 2 columns]
           thermax      थरमैक्स
0        sikhaaega      सिखाएगा
1            learn         लर्न
2         twitters     ट्विटर्स
3      tirunelveli  तिरुनेलवेली
4     independence  इंडिपेंडेंस
...            ...          ...
4090       saflata       सफ़लता
4091        shbana        शबाना
4092  khaatootolaa     खातूटोला
4093    shivastava     शिवास्तव
4094  preranapuree  प्रेरणापुरी

[4095 rows x 2 columns]
         jaisawal       जयसवाल
0           bajai         बजाई
1       sanghthan        संघठन
2         haiwaan        हैवान
3         nilgiri      नीलगिरि
4       drutgrami  द्रुतग्रामी
...           ...  

In [21]:
df_train.loc[0]

shastragaar     bindhya
शस्त्रागार     बिन्द्या
Name: 0, dtype: object

In [28]:
df_train.iloc[:,0]

0            bindhya
1          kirankant
2        yagyopaveet
3            ratania
4         vaganyache
            ...     
51194          toned
51195     mutanaazaa
51196      asahmaton
51197      sulgaayin
51198    anchuthengu
Name: shastragaar, Length: 51199, dtype: object

In [6]:
df_train = df_train[:]
df_valid = df_valid[:200]

In [4]:
eng_alphabets = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
pad_char = '-PAD-'

eng_alpha2index = {pad_char:0}
for index, alpha in enumerate(eng_alphabets):
  eng_alpha2index[alpha] = index+1

print(eng_alpha2index)

{'-PAD-': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10, 'K': 11, 'L': 12, 'M': 13, 'N': 14, 'O': 15, 'P': 16, 'Q': 17, 'R': 18, 'S': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26}


In [5]:
hindi_alphabets = [chr(alpha) for alpha in range(2304, 2432)]
hindi_alphabet_size = len(hindi_alphabets)

hindi_alpha2index = {pad_char : 0}
for index, alpha in enumerate(hindi_alphabets):
  hindi_alpha2index[alpha] = index + 1

print(hindi_alpha2index)

{'-PAD-': 0, 'ऀ': 1, 'ँ': 2, 'ं': 3, 'ः': 4, 'ऄ': 5, 'अ': 6, 'आ': 7, 'इ': 8, 'ई': 9, 'उ': 10, 'ऊ': 11, 'ऋ': 12, 'ऌ': 13, 'ऍ': 14, 'ऎ': 15, 'ए': 16, 'ऐ': 17, 'ऑ': 18, 'ऒ': 19, 'ओ': 20, 'औ': 21, 'क': 22, 'ख': 23, 'ग': 24, 'घ': 25, 'ङ': 26, 'च': 27, 'छ': 28, 'ज': 29, 'झ': 30, 'ञ': 31, 'ट': 32, 'ठ': 33, 'ड': 34, 'ढ': 35, 'ण': 36, 'त': 37, 'थ': 38, 'द': 39, 'ध': 40, 'न': 41, 'ऩ': 42, 'प': 43, 'फ': 44, 'ब': 45, 'भ': 46, 'म': 47, 'य': 48, 'र': 49, 'ऱ': 50, 'ल': 51, 'ळ': 52, 'ऴ': 53, 'व': 54, 'श': 55, 'ष': 56, 'स': 57, 'ह': 58, 'ऺ': 59, 'ऻ': 60, '़': 61, 'ऽ': 62, 'ा': 63, 'ि': 64, 'ी': 65, 'ु': 66, 'ू': 67, 'ृ': 68, 'ॄ': 69, 'ॅ': 70, 'ॆ': 71, 'े': 72, 'ै': 73, 'ॉ': 74, 'ॊ': 75, 'ो': 76, 'ौ': 77, '्': 78, 'ॎ': 79, 'ॏ': 80, 'ॐ': 81, '॑': 82, '॒': 83, '॓': 84, '॔': 85, 'ॕ': 86, 'ॖ': 87, 'ॗ': 88, 'क़': 89, 'ख़': 90, 'ग़': 91, 'ज़': 92, 'ड़': 93, 'ढ़': 94, 'फ़': 95, 'य़': 96, 'ॠ': 97, 'ॡ': 98, 'ॢ': 99, 'ॣ': 100, '।': 101, '॥': 102, '०': 103, '१': 104, '२': 105, '३': 106, '४': 107, '५': 108, '६': 109, '७': 

##pytorch tutorial

In [29]:

import torch
import math


dtype = torch.float
# device = torch.device("cpu")
device = torch.device("cuda:0") # Uncomment this to run on GPU

# Create random input and output data
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Randomly initialize weights
a = torch.randn((), device=device, dtype=dtype)
b = torch.randn((), device=device, dtype=dtype)
c = torch.randn((), device=device, dtype=dtype)
d = torch.randn((), device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of a, b, c, d with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Update weights using gradient descent
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d


print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

99 3991.02197265625
199 2800.90478515625
299 1967.4998779296875
399 1383.5478515625
499 974.1511840820312
599 686.981201171875
699 485.44427490234375
799 343.9365234375
899 244.5323028564453
999 174.674072265625
1099 125.55970764160156
1199 91.01603698730469
1299 66.71154022216797
1399 49.60511016845703
1499 37.561100006103516
1599 29.078697204589844
1699 23.102916717529297
1799 18.891891479492188
1899 15.923669815063477
1999 13.83096694946289
Result: y = 0.0733179822564125 + 0.8710255026817322 x + -0.012648571282625198 x^2 + -0.09536225348711014 x^3
